# Section 6.4.3: Automated Assistant QA System

This notebook demonstrates how to build an automated assistant using extractive and generative QA techniques.
- Extractive QA: Uses DistilBERT to locate answers from context.
- Generative QA: Uses T5 model to generate answers.
- Vector store: Uses ChromaDB with HuggingFace embeddings.

**Use case**: IT or HR assistant answering enterprise-level queries.

In [ ]:
# Install required libraries (Uncomment and run if not installed)
# !pip install transformers datasets chromadb langchain
# !pip install sentence-transformers


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import os

## Step 1: Prepare documents
Assume we have some IT/HR knowledge documents to index and query from.

In [ ]:
# Sample document creation
doc_text = """
Q: How to reset your corporate email password?
A: You can reset your password by visiting the internal IT portal and clicking on 'Reset Password'.

Q: What is the leave policy for new employees?
A: New employees are entitled to 20 days of annual leave, starting from their date of joining.

Q: Who do I contact for hardware issues?
A: Please reach out to the IT Helpdesk at helpdesk@company.com for hardware-related problems.
"""
with open("knowledge_base.txt", "w") as f:
    f.write(doc_text)

## Step 2: Create vector store from documents using HuggingFace embeddings

In [ ]:
# Load and split the document
loader = TextLoader("knowledge_base.txt")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

# Create embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(split_docs, embedding_model, persist_directory="chroma_store")
vectordb.persist()

## Step 3: Setup Extractive QA with DistilBERT

In [ ]:
# Load DistilBERT for QA
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
question = "How can I reset my email password?"
context = split_docs[0].page_content
result = qa_pipeline(question=question, context=context)
print("Extractive QA Answer:", result['answer'])

## Step 4: Setup Generative QA with T5 model

In [ ]:
# Load T5 for Generative QA
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

input_text = f"question: {question} context: {context}"
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
output_ids = t5_model.generate(input_ids, max_length=50)
answer = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generative QA Answer:", answer)

## Step 5: Retrieval-Augmented QA using Langchain + Chroma + T5

In [ ]:
from transformers import pipeline as hf_pipeline

qa_gen_pipe = hf_pipeline("text2text-generation", model="t5-small")
llm = HuggingFacePipeline(pipeline=qa_gen_pipe)
retriever = vectordb.as_retriever()

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
result = qa_chain.run("Who to contact for hardware issues?")
print("Hybrid QA Answer:", result)